In [13]:
# Imports
import sqlite3

In [14]:
# Instantiates the connection and the cursor to the database.
connection = sqlite3.connect("mecenact.db")
cursor = connection.cursor()

In [15]:
# Creates the table to store the users' information.
cursor.execute("""
            CREATE TABLE users (
            username TEXT,
            password TEXT NOT NULL,
            post_count INTEGER NOT NULL,
            PRIMARY KEY (username) )
            """)

In [16]:
# Creates the table to store the posts' information.
cursor.execute("""
            CREATE TABLE posts (
            author TEXT,
            id INTEGER,
            title TEXT NOT NULL,
            image BLOB NOT NULL,
            body TEXT,
            PRIMARY KEY (id, author),
            FOREIGN KEY (author) 
               REFERENCES users (username)
               ON DELETE CASCADE )
            """)

In [17]:
# Creates the table to store the comments' information.
cursor.execute("""
            CREATE TABLE tags (
            tag TEXT,
            author TEXT,
            post_id INTEGER,
            PRIMARY KEY (tag, author, post_id),
            FOREIGN KEY (author, post_id) 
               REFERENCES posts (author, id) 
               ON DELETE CASCADE )
            """)

In [18]:
# Commit any changes and close the connection.
connection.commit()
connection.close()